<a href="https://colab.research.google.com/github/LeeKLTW/tf1/blob/master/Predicting_Movie_Reviews_with_BERT_on_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0606 07:06:18.931993 139709332486016 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
! pip install bert_tensorflow

     |████████████████████████████████| 71kB 3.3MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [5]:
OUTPUT_DIR = 'OUTPUT_DIR_NAME'
try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
except:
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME *****


```
aclImdb
├── README
├── imdb.vocab
├── imdbEr.txt
├── test
│   ├── labeledBow.feat
│   ├── neg
│   ├── pos
│   ├── urls_neg.txt
│   └── urls_pos.txt
└── train
    ├── labeledBow.feat
    ├── neg
    ├── pos
    ├── unsup
    ├── unsupBow.feat
    ├── urls_neg.txt
    ├── urls_pos.txt
    └── urls_unsup.txt
  
```



In [0]:
from tensorflow import keras
import os
import re

def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    
    for file_path in os.listdir(directory):
        with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    data = pd.DataFrame.from_dict(data)
    return data

def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory,"pos"))
    neg_df = load_directory_data(os.path.join(directory,"neg"))
    pos_df['polarity'] = 1
    neg_df['polarity'] = 0
    data = pd.concat([pos_df, neg_df]).sample(frac=1.).reset_index(drop=True)
    return data
    
def download_and_load_datasets(force_download=False):
    dataset = keras.utils.get_file(fname="aclImdb.tar.gz",
                                   origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", extract=True)
    train_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "test"))
    return train_df, test_df
    


In [7]:
train_df, test_df = download_and_load_datasets()

84131840/84125825 [==============================] - 8s 0us/step


In [0]:
train_data = train_df.sample(5000)
test_data = test_df.sample(5000)

In [9]:
train_data.head()

,sentence,sentiment,polarity
7265,I thought the film could be a bit more complex...,7,1
9057,We know that firefighters and rescue workers a...,1,0
6676,"This is a good movie, but it is not recommende...",10,1
22193,I found this film embarrassing to watch. I fel...,1,0
20079,I have yet to see a film with Nolte in it that...,9,1


In [10]:
test_data.head()

,sentence,sentiment,polarity
13352,"I had heard about ""gaming"" and ""Dungeons and D...",9,1
3009,I have to admit I was deceived by the title an...,2,0
19758,This film proves you don't need a Hollywood bu...,9,1
12578,"Clouzot followed Le Corbeau, where no one knew...",9,1
20794,I am pretty surprised to see that this movie e...,1,0


In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
label_list = [0,1]

In [0]:
# bert.run_classifier.InputExample
train_input_example = train_data.apply(lambda x:bert.run_classifier.InputExample(guid=None, text_a = x[DATA_COLUMN], text_b=None,label=x[LABEL_COLUMN]),axis=1)
test_input_example = test_data.apply(lambda x:bert.run_classifier.InputExample(guid=None, text_a = x[DATA_COLUMN], text_b=None,label=x[LABEL_COLUMN]),axis=1)


In [13]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def create_tokenizer_from_hub_module():
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case =sess.run([tokenization_info["vocab_file"],tokenization_info["do_lower_case"]])
            
            return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()


Instructions for updating:
Colocations handled automatically by placer.


W0606 07:07:14.784930 139709332486016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 07:07:17.159623 139709332486016 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [14]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [17]:
MAX_SEQ_LENGTH = 128
train_features = bert.run_classifier.convert_examples_to_features(train_input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
# This function is not used by this file but is still used by the Colab and people who depend on it.

"""Convert a set of `InputExample`s to a list of `InputFeatures`."""
"""
guid
tokens
input_ids
input_mask
segment_ids
label
"""

INFO:tensorflow:Writing example 0 of 5000


I0606 07:09:14.219772 139709332486016 run_classifier.py:774] Writing example 0 of 5000


INFO:tensorflow:*** Example ***


I0606 07:09:14.233097 139709332486016 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0606 07:09:41.696974 139709332486016 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0606 07:09:41.699825 139709332486016 run_classifier.py:468] label: 0 (id = 0)


'\nguid\ntokens\ninput_ids\ninput_mask\nsegment_ids\nlabel\n'

In [0]:
# create model
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    
    bert_inputs = dict(input_ids=input_ids, input_mask= input_mask, segment_ids=segment_ids)
    
    bert_outputs = bert_module(inputs=bert_inputs, signature='tokens',as_dict=True)
    
    output_layer = bert_outputs['pooled_output']
    
    hidden_size = output_layer.shape[-1].value
    
    output_weights = tf.get_variable('output_weights', [num_labels,hidden_size],initializer=tf.truncated_normal_initializer(stddev=0.02))
    output_bias = tf.get_variable('output_bias', [num_labels],initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    # define loss function
    with tf.variable_scope("loss"):
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        
        logits = tf.matmul(output_layer,output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits,output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1) #logit not logarithmic
        
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        
        if is_predicting:
            return (predicted_labels, log_probs)
        
        per_example_loss = tf.reduce_sum(one_hot_labels*log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss,predicted_labels, log_probs)

In [0]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,num_warmup_steps):
    def model_fn(features, labels, mode, params):
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]


        
        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
        
        if not is_predicting:
            (loss,predicted_labels,log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
            
            train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
            
            def metric_fn(label_ids, predicted_labels):
                
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels)
                true_positive = tf.metrics.true_positives(label_ids, predicted_labels)
                true_negative = tf.metrics.true_negatives(label_ids, predicted_labels)
                false_positive = tf.metrics.false_positives(label_ids, predicted_labels)
                false_negative = tf.metrics.false_negatives(label_ids, predicted_labels)
                return {"accuracy":accuracy,
                       "f1_score":f1_score,
                       "auc": auc,
                       "precision":precision,
                       "true_positive":true_positive,
                       "true_negative":true_negative,
                       "false_positive": false_positive,
                       "false_negative": false_negative
                       }            
            eval_metrics = metric_fn(label_ids, predicted_labels)
            
            if mode == tf.estimator.ModeKeys.TRAIN:
                
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        else:
            (loss,predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels)
            predictions = {'probabilities':log_probs, 'labels':predicted_labels}
    
    return model_fn

In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAINING_EPOCHS = 10

# warm up

WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINT_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
num_train_steps = int(len(train_features)/ BATCH_SIZE * NUM_TRAINING_EPOCHS)
num_warmup_steps = int(num_train_steps* WARMUP_PROPORTION)

In [0]:
run_config = tf.estimator.RunConfig(model_dir=OUTPUT_DIR, 
                                    save_summary_steps=SAVE_SUMMARY_STEPS,
                                    save_checkpoints_steps= SAVE_CHECKPOINT_STEPS)

In [23]:
model_fn = model_fn_builder(num_labels=len(label_list), 
                            learning_rate=LEARNING_RATE, 
                            num_train_steps=num_train_steps, 
                            num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(model_fn=model_fn, 
                                   config=run_config,
                                   params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f10637f62b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0606 07:10:26.452908 139709332486016 estimator.py:201] Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f10637f62b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0606 07:11:00.878042 139709332486016 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 07:11:04.435531 139709332486016 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

I0606 07:11:26.917134 139709332486016 session_manager.py:493] Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into OUTPUT_DIR_NAME/model.ckpt.


I0606 07:11:38.325304 139709332486016 basic_session_run_hooks.py:594] Saving checkpoints for 0 into OUTPUT_DIR_NAME/model.ckpt.


INFO:tensorflow:loss = -0.7185857, step = 1


I0606 07:12:42.549192 139709332486016 basic_session_run_hooks.py:249] loss = -0.7185857, step = 1


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [139]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0606 07:02:44.565648 139892431648640 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 07:02:49.202524 139892431648640 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 468: accuracy = 0.5028, auc = 0.5, f1_score = 0.6691508, false_negative = 0.0, false_positive = 2486.0, global_step = 468, loss = -15.961964, precision = 0.5028, true_negative = 0.0, true_positive = 2514.0


I0606 07:03:51.153890 139892431648640 estimator.py:1979] Saving dict for global step 468: accuracy = 0.5028, auc = 0.5, f1_score = 0.6691508, false_negative = 0.0, false_positive = 2486.0, global_step = 468, loss = -15.961964, precision = 0.5028, true_negative = 0.0, true_positive = 2514.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 468: OUTPUT_DIR_NAME/model.ckpt-468


I0606 07:03:54.678534 139892431648640 estimator.py:2039] Saving 'checkpoint_path' summary for global step 468: OUTPUT_DIR_NAME/model.ckpt-468


{'accuracy': 0.5028,
 'auc': 0.5,
 'f1_score': 0.6691508,
 'false_negative': 0.0,
 'false_positive': 2486.0,
 'global_step': 468,
 'loss': -15.961964,
 'precision': 0.5028,
 'true_negative': 0.0,
 'true_positive': 2514.0}